# Documento de análisis de métricas

En este documento de Jupyter Notebook vamos a analizar las métricas propuestas en el **Documento de investigación**

Para ello se va a empezar uniendo todos los archivos .json que hay en la carpeta **./JSONS** en un solo .json, tras ello se harán los cálculos correspondientes.

In [ ]:
import pandas as pd
import os
import json
import numpy as np

In [ ]:
folder="./JSONS"
combinedData=[]
for file in os.listdir(folder):
    if(file.endswith('.json')):
        route=os.path.join(folder,file)
        with open(route,'r')as f:
            jsonfile=json.load(f)
            combinedData.extend(jsonfile['data'])
jsonFile={'data':combinedData}
with open('data_combined.json', 'w') as f:
    json.dump(jsonFile, f)

In [ ]:
x = pd.json_normalize(jsonFile['data'])
groupedEvents = x.groupby('eventType')
##for eid, group in groupedEvents:
    ##display(group)
    ##display(eid)

### Los eventos y sus ID:
```C#
//Eventos comunes
 SessionStart = 0
 SessionStop = 1
 GameStart = 2
 GameEnd = 3
 LevelStart = 4
 LevelEnd = 5
 Pause = 6
 Resume = 7
 // Eventos de juego
 Attack = 8
 EnemyReceive = 9
 PlayerReceive = 10
 PlayerDead = 11
 RoomMove = 12
```


# Métrica 1: Tiempo de finalización de nivel
¿El jugador tarda demasiado tiempo en completar cada uno de los dos niveles?

In [ ]:
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
## estoy usando los de la sesion por ahora, pero se haria con los de cada nivel.
startEvent = groupedEvents.get_group(4)
#display(startEvent)
endEvent = groupedEvents.get_group(5)
#display(endEvent)
     
endEvent_filtered = endEvent[endEvent['LevelEnd'] == 0]


startEvent_filtered = startEvent[startEvent['Level'].isin(endEvent_filtered['Level']) & startEvent['GameSession'].isin(endEvent_filtered['GameSession'])]
endEvent_filtered = endEvent_filtered.copy()

endEvent_filtered.loc[:, 'combined'] = endEvent_filtered['Level'].astype(str) + '-' + endEvent_filtered['GameSession'].astype(str)
startEvent = startEvent.copy()

startEvent.loc[:, 'combined'] = startEvent['Level'].astype(str) + '-' + startEvent['GameSession'].astype(str)


startEvent_filtered = startEvent[startEvent['combined'].isin(endEvent_filtered['combined'])]


del startEvent_filtered['combined']
del endEvent_filtered['combined']
#print("Eventos de inicio de niveles completados:\n")
#display(startEvent_filtered)

#print("Eventos de fin de niveles completados:\n")
#display(endEvent_filtered) 

durations = []
for i in range(len(startEvent_filtered)):
    start_time = startEvent_filtered.iloc[i]['timestamp']
    end_time = endEvent_filtered.iloc[i]['timestamp']
    total_pause_duration = 0
    if 6 in groupedEvents.groups and 7 in groupedEvents.groups:
        pauseEvents = groupedEvents.get_group(6)
        resumeEvents = groupedEvents.get_group(7)
        #Filtrar pausas
        
        pauses_in_range = pauseEvents[(pauseEvents['timestamp'] >= start_time) & (pauseEvents['timestamp'] <= end_time)]
        resumes_in_range = resumeEvents[(resumeEvents['timestamp'] >= start_time) & (resumeEvents['timestamp'] <= end_time)]
        #if not pauses_in_range.empty:
            #print("Eventos de pausa:")
            #display(pauses_in_range)
            
        #if not resumes_in_range.empty:
            #print("Eventos de reanudación:")
            #display(resumes_in_range)
        #Sumar tiempo pausa
        for j in range(len(pauses_in_range)):
            pause_time = pauses_in_range.iloc[j]['timestamp']
            resume_time = resumes_in_range.iloc[j]['timestamp']
            pause_duration = resume_time - pause_time
            total_pause_duration += pause_duration
            
    a_seconds = start_time / 1000
    a_prueba= str(timedelta(seconds=int(a_seconds)))
    b_seconds = end_time / 1000
    total_pause_duration= total_pause_duration / 1000
    duration = b_seconds - a_seconds- total_pause_duration
    durations.append(duration)

print("Duraciones de todos los eventos: ")
print(durations)

startEvent_filtered = startEvent_filtered.copy()
startEvent_filtered['duration'] = durations

#Separar los eventos por nivel
level1_events = startEvent_filtered[startEvent_filtered['Level'] == 1]
level2_events = startEvent_filtered[startEvent_filtered['Level'] == 2]

#Media de las duraciones para cada nivel
mean_duration_level1 = level1_events['duration'].mean()
mean_duration_level2 = level2_events['duration'].mean()
#Mediana de las duraciones de cada nivel
median_duration_level1 = level1_events['duration'].median()
median_duration_level2 = level2_events['duration'].median()
max_duration_level1 = level1_events['duration'].max()
max_duration_level2 = level2_events['duration'].max()

maxi = max(max_duration_level1, max_duration_level2)
print("Media de las duraciones del nivel 1: ", mean_duration_level1)
print("Media de las duraciones del nivel 2: ", mean_duration_level2)
print("Mediana de las duraciones del nivel 1: ", median_duration_level1)
print("Mediana de las duraciones del nivel 2: ", median_duration_level2)
#Histograma Nivel 1
plt.hist(level1_events['duration'], bins=10, range=(0,maxi), density=True, edgecolor='black')
plt.title('Nivel 1')
plt.xlabel('Duración (segundos)')
plt.ylabel('Frecuencia')
plt.show()
#Histograma Nivel 2
plt.hist(level2_events['duration'], bins=10, edgecolor='black')
plt.title('Nivel 2')
plt.xlabel('Duración (segundos)')
plt.ylabel('Frecuencia')
plt.show()
del startEvent_filtered['duration']

##dt_object_start = datetime.fromtimestamp(a_seconds)
##dt_object_end = datetime.fromtimestamp(b_seconds)

##print("Start time: ", dt_object_start)
##print("End time: ", dt_object_end)
    

# Métrica 2: Cantidad de movimientos por nivel
¿El jugador usa demasiados movimientos de salas en cada uno de los dos niveles?

In [ ]:
#Eventos de movimiento de sala
room_moves = groupedEvents.get_group(12)

#Agrupar los movimientos por sesión y nivel
movements = room_moves.groupby(['GameSession', 'Level']).size().reset_index(name='Movements')

#Obtener las sesiones únicas de los eventos de fin
endEventSessions = endEvent_filtered['GameSession'].unique()

#Filtrar los movimientos que pertenecen a una sesión de juego completada
completed_movements = movements[movements['GameSession'].isin(endEventSessions)]

#LevelIDs de los eventos completados
completed_levels = endEvent_filtered[endEvent_filtered['LevelEnd'] == 0]['Level'].unique()

print("Movimientos de sesión completados por niveles:\n")
display(completed_movements)

#Level 1
filtered_movements_1 = completed_movements[completed_movements['Level'] == 1.0]
conteos_nivel_1 = {}

# Iterar sobre las filas del DataFrame filtered_movements_1
for index, fila in filtered_movements_1.iterrows():
    movimientos = fila['Movements']
    clave=int(movimientos/5)
    clave= clave*5
    if clave in conteos_nivel_1:
        conteos_nivel_1[clave]=conteos_nivel_1[clave]+1
    else:
        conteos_nivel_1[clave]=1
print(conteos_nivel_1)
 # Obtener las claves y valores del diccionario
claves = list(conteos_nivel_1.keys())
valores = list(conteos_nivel_1.values())

# Crear el gráfico de barras
plt.bar(claves, valores)
plt.xlabel('Inicio intervalos')
plt.ylabel('Cantidad de veces')
plt.title('Moda de movimientos de sala en nivel 1')

#Esto permite ver mejor los valores pero si son muy dispersos en vez de ayudar dificulta
plt.yticks(range(int(max(valores)) + 1))
ubicaciones_x = range(0, max(claves) + 1, 5)
etiquetas_x = [str(i) for i in ubicaciones_x]
plt.xticks(ubicaciones_x)
# Mostrar el gráfico
plt.show() 

#Level 2
filtered_movements_2 = completed_movements[completed_movements['Level'] == 2.0]
conteos_nivel_2 = {}

# Iterar sobre las filas del DataFrame filtered_movements_1
for index, fila in filtered_movements_2.iterrows():
    movimientos = fila['Movements']
    #obtenemos la 
    clave=int(movimientos/5)
    clave=clave*5
    if clave in conteos_nivel_2:
        conteos_nivel_2[clave]=conteos_nivel_2[clave]+1
    else:
        conteos_nivel_2[clave]=1
 # Obtener las claves y valores del diccionario
claves = list(conteos_nivel_2.keys())
valores = list(conteos_nivel_2.values())
print(claves)
# Crear el gráfico de barras
plt.bar(claves, valores)
plt.xlabel('Inicio intervalos')
plt.ylabel('Cantidad de veces')
plt.title('Moda de movimientos de sala en nivel 2')

#Esto permite ver mejor los valores pero si son muy dispersos en vez de ayudar dificulta
plt.yticks(range(int(max(valores)) + 1))
ubicaciones_x = range(0, max(claves) + 1, 5)
etiquetas_x = [str(i) for i in ubicaciones_x]
plt.xticks(ubicaciones_x)
# Mostrar el gráfico
plt.show()

#QUITAMOS LAS MEDIAS???????????????????????????????????????????????????????????????????????????
mean_movements_1 = filtered_movements_1['Movements'].mean()
mean_movements_2 = filtered_movements_2['Movements'].mean()

print(f"La media de movimientos de sala para el nivel 1 es: {mean_movements_1}")
print(f"La media de movimientos de sala para el nivel 2 es: {mean_movements_2}")

# Métrica 3: Precisión de golpes a los enemigos
¿El jugador falla significativamente más ataques en el nivel 2 que en el nivel 1?

In [ ]:
# FALTA AGRUPAR POR NIVELES
# Verificar si los grupos existen antes de acceder a ellos
if 8 in groupedEvents.groups:
    attack = groupedEvents.get_group(8)
    enemy_receive = groupedEvents.get_group(9) if 9 in groupedEvents.groups else None
    #display(attack)
    #display(enemy_receive)

    # Se agrupan los ataques realizados y los recibidos por los enemigos y se cuenta el número de cada uno
    attack_count = attack.groupby('Level').size().reset_index(name='Total Attacks')
    
    # Si el grupo 9 existe, contamos los aciertos; de lo contrario, asumimos que son 0
    if enemy_receive is not None:
        hit_count = enemy_receive.groupby('Level').size().reset_index(name='Total Hits')
    else:
        hit_count = pd.DataFrame({'Level': attack_count['Level'], 'Total Hits': 0})
    
    # Agrupar los datos en un único dataframe
    accuracy = pd.merge(attack_count, hit_count, on='Level', how='left')
    accuracy['Total Hits'] = accuracy['Total Hits'].fillna(0)

    # Calcular la precisión de los golpes
    accuracy['Accuracy'] = (accuracy['Total Hits'] / accuracy['Total Attacks']) * 100
    
    import matplotlib.pyplot as plt

    # Assuming 'accuracy' is your DataFrame
    accuracy['Level'] = accuracy['Level'].astype(int)  # Convert Level to integer

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.bar(accuracy['Level'], accuracy['Accuracy'], color='skyblue')
    plt.xlabel('Level', fontsize=14)
    plt.ylabel('Hit Accuracy (%)', fontsize=14)
    plt.title('Hit Accuracy per Level', fontsize=16)
    plt.xticks(accuracy['Level'])
    plt.ylim(0, 100)  # Set the y-axis range from 0 to 100
    plt.show()
    
    accuracy['Accuracy'] = accuracy['Accuracy'].apply(lambda x: f"{x:.2f}%")
    print("Hit accuracy:\n")
    print(accuracy[['Level', 'Accuracy']].to_string(index=False))

else:
    print("Nunca se han llevado a cabo ataques.")


# Métrica 4: Impactos de los elementos dañinos
Se estudiará la distribución del número de veces que cada elemento dañino(enemigos, sierra, alcantarilla) impacta al jugador. Se calculará el porcentaje de impactos de cada obstáculo con respecto al total de impactos.


In [ ]:
player_receive = groupedEvents.get_group(10)

# Se agrupan los impactos por sesión y tipo de enemigo y se cuenta el número de cada una
impact_count = player_receive.groupby(['Level', 'EnemyType']).size().reset_index(name='Impacts')

# Impactos totales por sesión
total_impact_count = player_receive.groupby('Level').size().reset_index(name='Total Impacts')

# Agrupar los datos en un único dataframe
impact_count = impact_count.merge(total_impact_count, on='Level', how='left')

# Calcular el porcentaje de impactos provocados por cada tipo de enemigo
impact_count['Accuracy'] = (impact_count['Impacts'] / impact_count['Total Impacts']) * 100

# Mapeo de nombres de enemigos
enemy_names = {
    0: "Robot",
    1: "Spider",
    2: "Saw",
    3: "Sewer"
}
impact_count['EnemyType'] = impact_count['EnemyType'].map(enemy_names)

levels = impact_count['Level'].unique()

# Generar un gráfico para cada nivel
for level in levels:
    # Filtrar los datos para el nivel actual
    data_for_pie = impact_count[impact_count['Level'] == level]
    
    plt.figure(figsize=(8, 8))
    plt.pie(data_for_pie['Accuracy'], labels=data_for_pie['EnemyType'].astype(str), autopct='%1.1f%%', startangle=140)
    plt.title(f'Porcentaje de Impactos por Tipo de Enemigo en el Nivel {level}')
    plt.show()
    
impact_count['Accuracy'].fillna(0, inplace=True)
impact_count['Accuracy'] = impact_count['Accuracy'].apply(lambda x: f"{x:.3f}%")

print("Enemy impacts:\n")
display(impact_count[['Level', 'EnemyType', 'Impacts', 'Accuracy']])
print("\nTotal impacts: ", impact_count['Impacts'].sum())